In [1]:
import requests
import json

In [2]:
BASE_URL = "https://search-api.epfl.ch/api/unit?hl=en&showall=0&siteSearch=unit.epfl.ch&acro="

res = requests.get(BASE_URL + 'ASSOCIATIONS')
groups = json.loads(res.text)['subunits']

units = []
for i, group in enumerate(groups):
    res = requests.get(BASE_URL + group['acronym'])
    group_units = json.loads(res.text)['subunits']
    groups[i] = {
        **group,
        'id': i
    }
    for unit in group_units:
        units.append({
            'group_name': group['acronym'],
            'group_id': i,
            **unit
        })

In [3]:
for i, unit in enumerate(units):
    units[i] = {
        **unit,
        'id': i,
        'label': unit['acronym'],
        'type': 'unit'
    }

In [30]:
with open("nodes.json", "w", encoding='utf8') as outfile:
    json.dump(units, outfile, ensure_ascii=False)
with open("groups.json", "w", encoding='utf8') as outfile:
    json.dump(groups, outfile, ensure_ascii=False)

In [4]:
from ldap3 import Server, Connection, SUBTREE
c = Connection("ldaps://ldap.epfl.ch:636", auto_bind=True)
c.bind()

accreds = []
for unit in units:
    c.search(search_base = 'o=ehe,c=ch',
            search_filter = f"(&(ou={unit['acronym']})(objectClass=person))",
            search_scope = SUBTREE,
            attributes = '*')

    results = c.response
    for user in results:
        user = dict(user['attributes'])
        accreds.append({
            'sciper': int(user['uniqueIdentifier'][0]),
            'name': user['displayName'],
            'unit_name': unit['acronym'],
            'unit_id': unit['id']
        })


In [18]:
unit_size = dict()
for accred in accreds:
    unit_id = accred['unit_id']
    if(unit_id in unit_size):
        unit_size[unit_id] += 1
    else:
        unit_size[unit_id] = 1

for i, unit in enumerate(units):
    if (unit['id'] not in unit_size):
        size = 0
    else:
        size = unit_size[unit['id']]
    units[i] = {
        **unit,
        'size': size
    }

In [19]:
n_accreds = dict()
for accred in accreds:
    if (accred['sciper'] in n_accreds):
        n_accreds[accred['sciper']] += 1
    else:
        n_accreds[accred['sciper']] = 1

users = []
for accred in accreds:
    if (n_accreds[accred['sciper']] > 1):
        user = {
            'id': accred['sciper'],
            'name': accred['name'],
            'type': 'user',
            'accreds': n_accreds[accred['sciper']]
        }
        if (user not in users):
            users.append(user)

        

In [31]:
with open("users.json", "w", encoding='utf8') as outfile:
    json.dump(users, outfile, ensure_ascii=False)

In [20]:
links = []
for i, accred in enumerate(accreds):
    for unit in units:
        if (unit['acronym'] == accred['unit_name']):
            unit_id = unit['id']
    for user in users:
        if (user['id'] == accred['sciper']):
            user_id = user['id']
            links.append({
                'target': unit_id,
                'source': user_id
            })

In [29]:
with open("links.json", "w", encoding='utf8') as outfile:
    json.dump(links, outfile, ensure_ascii=False)

In [21]:
data = {
    'nodes': units + users,
    'links': links
}

In [20]:
for node in data['nodes']:
    if (node['id'] == 288450):
        print(node)

{'id': 288450, 'name': 'Benjamin Bruno Vincent Louis', 'accreds': 2}


In [23]:
for link in data['links']:
    if (link['source'] == 288450):
        print(link)

{'target': 0, 'source': 288450}
{'target': 49, 'source': 288450}


In [9]:
with open("data.json", "w", encoding='utf8') as outfile:
    json.dump(data, outfile, ensure_ascii=False)